In [855]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Regressors
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#Metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
%matplotlib inline
#Clustering
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
#Otros
from sklearn.multioutput import MultiOutputRegressor

### CSV Loading

In [485]:
dataset = pd.read_csv('./data/t_data/anime_cleaned_treated_v2_1.csv')

In [486]:
dataset.head()

,Genres,Source,Type,Episodes,Decimal,Romance,Shoujo,Ecchi,Shounen,Fantasy,...,Mistery,Thriller,Supernatural,Gore,SciFi,Action,Otro,Popularity,Score,Favorites
0,"Comedy, Romance, Shounen, Supernatural",6,5,12,147,1,0,0,1,0,...,0,0,1,0,0,0,0,231,7.63,2809
1,"Comedy, Romance, Shounen",6,5,26,19,1,0,0,1,0,...,0,0,0,0,0,0,0,366,7.89,2579
2,"Comedy, Romance, Fantasy, Shoujo",6,5,51,519,1,1,0,0,1,...,0,0,0,0,0,0,0,1173,7.55,802
3,"Comedy, Romance, Fantasy",9,5,38,7,1,0,0,0,1,...,0,0,0,0,0,0,0,916,8.21,3344
4,"Comedy, Romance, Shounen",6,5,25,19,1,0,0,1,0,...,0,0,0,0,0,0,0,426,8.67,2082


### Training

In [851]:
data = dataset.values
X = data[:int(len(data)*0.7),1:22]
y = data[:int(len(data)*0.7),22:24]
X_t = data[int(len(data)*0.7):,1:22]
y_t = data[int(len(data)*0.7):,22:24]
print y[0]

[231 7.63]


In [856]:
DT_R = DecisionTreeRegressor(max_depth=5)
RF_R = RandomForestRegressor(n_estimators=40)
GB_R = MultiOutputRegressor(GradientBoostingRegressor())
#np.array([6,5,12,147]).reshape(1,-1)

In [857]:
DT_R.fit(X,y)
RF_R.fit(X,y)
GB_R.fit(X,y)

MultiOutputRegressor(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
           n_jobs=1)

In [847]:
for i in range(10):
    print "============="
    print RF_R.predict(X_t[i].reshape(1,-1))
    print DT_R.predict(X_t[i].reshape(1,-1))
    print GB_R.predict(X_t[i].reshape(1,-1))
    print y_t[i]

[ 1809.39166667]
[ 3231.96540362]
[ 2334.3658176]
40
[ 4801.64]
[ 5715.97260274]
[ 5382.30646749]
3370
[ 4010.55]
[ 3325.4379562]
[ 3713.54253324]
5068
[ 2530.86666667]
[ 2011.57080132]
[ 1784.57310886]
2088
[ 9529.53041667]
[ 7203.13432836]
[ 7800.29689645]
10312
[ 8188.19791667]
[ 7203.13432836]
[ 6250.39246821]
9390
[ 3095.8]
[ 3913.33333333]
[ 3959.41475762]
1621
[ 1368.99166667]
[ 2011.57080132]
[ 617.57260891]
540
[ 398.13541667]
[ 3231.96540362]
[ 1387.48239655]
1264
[ 4082.2]
[ 3325.4379562]
[ 4209.10730399]
3683


In [858]:
s = cross_val_score(DT_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 2207.564 (+/- 691.70861)


In [859]:
s = cross_val_score(RF_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 2054.541 (+/- 685.06631)


In [860]:
s = cross_val_score(GB_R, X_t, y_t, scoring=make_scorer(mean_squared_error))
print "MSE %.3f (+/- %.5f)"%(np.sqrt(np.mean(s)), np.sqrt(np.std(s)))

MSE 1979.930 (+/- 788.60557)


### Pruebas de predicciones arbitrarias

In [924]:
new_anime = np.array([6,5,24,50822,1,1,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0]).reshape(1,-1)

In [929]:
GB_R.predict(new_anime)

array([[ 2545.09202523,     6.90995925]])